# Pattern Associator

We recreate the network of the [Pattern Associator tutorial from the CECN1 notebook](https://grey.colorado.edu/CompCogNeuro/index.php/CECN1_Pattern_Associator) exploring how the delta rule works and behave. 

In [ ]:
import numpy as np
import dotdot
import leabra
import graphs

In [ ]:
input_layer  = leabra.Layer(4)
output_spec  = leabra.LayerSpec(kwta_pct=1.0) # no inhibition
output_layer = leabra.Layer(2, spec=output_spec)

In [ ]:
conspec = leabra.ConnectionSpec(proj="Full", lrule='delta')
conn    = leabra.Connection(input_layer, output_layer, spec=conspec)

In [ ]:
network = leabra.Network(layers=[output_layer], connections=[conn])

In [ ]:
def event(k, network):
    """Run a minus phase and a plus phase for a given input/output pair"""
    inputs  = [0.0, 0.0, 0.0, 0.0]
    outputs = [0.0, 0.0]
    inputs[k] = 1.0
    outputs[int(k/2)] = 1.0  # desired output
    
    # minus phase
    input_layer.set_activities(inputs)
    network.settle()
    network.end_minus_phase()
    error = sum((np.array(output_layer.activities) - outputs)**2) 

    # plus phase: the output is set directly
    for t in range(network.spec.settle):
        output_layer.set_activities(outputs)
        network.cycle()
    output_layer.set_activities(outputs)
    network.end_plus_phase()
    
    return error

In [ ]:
conn.weights

In [ ]:
def trial():
    sse = 0.0
    sse += event(0, network)
    sse += event(1, network)
    sse += event(2, network)
    sse += event(3, network)
    return sse / 4

In [ ]:
err = [trial() for _ in range(20)]

In [ ]:
conn.weights

In [ ]:
graphs.line(range(1000), err, "Average error over trials", width=600)